In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
device = (
"cuda"
if torch.cuda.is_available()
else "mps"
if torch.backends.mps.is_available()
else "cpu")
print(f"Using {device} device")

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten= nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1,28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1397,  0.2041,  0.6017,  0.0991, -0.2483,  0.0127,  0.1236, -0.0320,
         -0.0806, -0.3894,  0.3511,  0.0344, -0.4932, -0.0810,  0.4838,  0.7067,
         -0.0969, -0.1458, -0.3223, -0.0055],
        [ 0.4161, -0.1018,  0.6614, -0.1041, -0.3544,  0.0488,  0.0014,  0.0260,
         -0.1830, -0.4037,  0.2263, -0.2408, -0.2935, -0.2253,  0.7677,  0.5747,
          0.0784, -0.1931, -0.3828,  0.1577],
        [ 0.5159,  0.0839,  0.5854, -0.1203, -0.5700, -0.0169,  0.1876,  0.4040,
         -0.3614, -0.4254,  0.3469,  0.1382, -0.4709, -0.5435,  0.5442,  0.2009,
         -0.1098,  0.0834, -0.6639, -0.0460]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1397, 0.2041, 0.6017, 0.0991, 0.0000, 0.0127, 0.1236, 0.0000, 0.0000,
         0.0000, 0.3511, 0.0344, 0.0000, 0.0000, 0.4838, 0.7067, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.4161, 0.0000, 0.6614, 0.0000, 0.0000, 0.0488, 0.0014, 0.0260, 0.0000,
         0.0000, 0.2263, 0.0000, 0.0000, 0.0000, 0.76

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0353,  0.0279,  0.0094,  ..., -0.0207, -0.0116, -0.0187],
        [ 0.0182, -0.0086, -0.0085,  ..., -0.0221, -0.0010, -0.0153]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0344, -0.0196], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0229,  0.0411, -0.0425,  ..., -0.0347, -0.0317, -0.0277],
        [ 0.0404, -0.0275,  0.0136,  ..., -0.0358, -0.0346,  0.0380]],
       device='cuda:0', grad_fn=<Sl